### HTML Table to Dataframe Converter (Beautifulsoup)

<br>

### Development Environment

In [1]:
import re
import requests
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup

### Page Information

In [2]:
def make_page_soup(link):
    target = link
    response = requests.get(target)
    response.raise_for_status()
    response.encoding='EUC-KR'
    page = str(response.content)
    soup = BeautifulSoup(response.text, 'html.parser') 
    return page, soup

In [3]:
link = "http://web.stanford.edu/class/cs224n/"
page, soup = make_page_soup(link)
tables = soup.find_all('table', attrs={'class':'table'})
table = tables[1]

### HTML Table to Dataframe Converter

In [24]:
def parse_by_pattern(text):
    link_list = []
    final_link_list = []
    href_pattern = re.findall(r'href=[\'"]?([^\'" >]+)', text)
    if href_pattern != None:
        number_of_link = len(href_pattern)
        for i in range(number_of_link):
            link = href_pattern[i]
            link_list.append(link)
        for link in link_list:
            if "slides/" in link or "project/" in link or \
                "readings/" in link or 'assignments/' in link:
                link = "http://web.stanford.edu/class/cs224n/" + link
                final_link_list.append(link)
    html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
    text = re.sub(html_pattern, ' ', text) 
    try:
        if len(final_link_list) > 0:
            for link in final_link_list:
                text +=  "\n" + link
        else:
            for link in link_list:
                text +=  "\n" + link
    except:
        pass  
    text = text.strip()
    text += "\n\n"
    return text

In [46]:
date_list = []
slides_tutorial_list = []
readings_list = []
assignment_list = []

table_tr = table('tr')
for i in range(len(table_tr)):
    if i > 0:
        try:
            date = str(table_tr[i]('td')[0])            
            date = parse_by_pattern(date)
        except:
            date = None        
        try:
            slides_tutorial = str(table_tr[i]('td')[1])
            slides_tutorial = parse_by_pattern(slides_tutorial)
        except:
            slides_tutorial = None
        try:    
            readings = str(table_tr[i]('td')[2])
            readings = parse_by_pattern(readings)
        except:
            readings = None
        try:   
            assignment = str(table_tr[i]('td')[3])
            assignment = parse_by_pattern(assignment)
        except:
            assignment = None

        date_list.append(date)
        slides_tutorial_list.append(slides_tutorial)
        readings_list.append(readings)
        assignment_list.append(assignment)  

In [54]:
slides_tutorial_df = pd.DataFrame({'date':date_list, 'content':slides_tutorial_list})
readings_df = pd.DataFrame({'date':date_list, 'content':readings_list})
assignment_df = pd.DataFrame({'date':date_list, 'content':assignment_list})

In [55]:
slides_tutorial_df.to_csv("slides_tutorial.txt")
readings_df.to_csv("readings.txt")
assignment_df.to_csv("assignment.txt")

In [56]:
slides_tutorial_df.to_excel("slides_tutorial.xlsx")
readings_df.to_excel("readings.xlsx")
assignment_df.to_excel("assignment.xlsx")

In [57]:
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_colwidth', 180)

In [58]:
slides_tutorial_df

,date,content
0,Tue Jan 10 Week 1\n\n,Word Vectors (by John Hewitt) \n \n [ slides ]\n video ]-->\n [ notes ]\n \n Gensim word vectors example:\n \n [ code ]\n [ preview...
1,Thu Jan 12\n\n,"Word Vectors, Word Window Classification, Language Models\n \n [ slides ]\n video ]-->\n [ notes ]\n \nhttp://web.stanford.edu/class/cs224n/slides/cs..."
...,...,...
27,Sat Mar 18\n\n,\n\n
28,Monday Mar 20\n\n,Poster Session\n \n [ code ]\n [ preview ]-->\n \nhttp://web.stanford.edu/class/cs224n/readings/cs224n-python-review-code-updated.zip\nhttp://web.stanford.edu/c...


In [59]:
readings_df

,date,content
0,Tue Jan 10 Week 1\n\n,Suggested Readings:\n \n Efficient Estimation of Word Representations in Vector Space (original word2vec paper) \n Distributed Representations of Words and Phrases and...
1,Thu Jan 12\n\n,Suggested Readings:\n \n GloVe: Global Vectors for Word Representation (original GloVe paper) \n Improving Distributional Similarity with Lessons Learned from Word Emb...
...,...,...
27,Sat Mar 18\n\n,\n\n
28,Monday Mar 20\n\n,5pm-9pm [ More details ] \n Location: Tressider Oak Lounge\n \nproject.html\n\n


In [60]:
assignment_df

,date,content
0,Tue Jan 10 Week 1\n\n,Assignment 1 out \n \n [ code ]\n \n [ preview ]\n \nhttp://web.stanford.edu/class/cs224n/assignments/a1.zip\nhttp://web.stanford.edu/class/cs224n/assi...
1,Thu Jan 12\n\n,\n\n
...,...,...
27,Sat Mar 18\n\n,\n\n
28,Monday Mar 20\n\n,\n\n


<br>

### Reference

<br><b>MOOC<b>
<br>[CS224N: Natural Language Processing with Deep Learning, Stanford University](http://web.stanford.edu/class/cs224n/)